## Preprocessing

In [1]:
# Import our dependencies
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis = 1, inplace = True)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column.
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df[application_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_types_counts = application_types_counts=application_df.APPLICATION_TYPE.value_counts()
application_types_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_types_counts[application_types_counts<500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = classification_counts=application_df.CLASSIFICATION.value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_above1 = classification_counts[classification_counts>1]
classification_counts_above1

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts<500].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_data = pd.get_dummies(application_df,dtype=float)
categorical_data.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Split our preprocessed data into our features and target arrays
y = categorical_data['IS_SUCCESSFUL'].values

# drop 'IS SUCCESSFUL'
X = categorical_data.drop('IS_SUCCESSFUL', axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 50)

categorical_data

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,1,5000,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,1,5000,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Importing Dependencies needed for "softmax activation function" section
from math import exp

In [13]:
# softmax activation function
def softmax(x):
 return exp(x) / exp(x).sum()

# linear activation function
def linear(x):
 return x

# tanh activation function
def tanh(x):
 return (exp(x) - exp(-x)) / (exp(x) + exp(-x))

### Attempt 1

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 5
hidden_nodes_layer2 = 10
hidden_nodes_layer3 = 15
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third Hidden Layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 225       
                                                                 
 dense_1 (Dense)             (None, 10)                60        
                                                                 
 dense_2 (Dense)             (None, 15)                165       
                                                                 
 dense_3 (Dense)             (None, 1)                 16        
                                                                 
Total params: 466 (1.82 KB)
Trainable params: 466 (1.82 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train, batch_size=128, epochs=75)

Epoch 1/75
201/201 [==============================] - 1s 900us/step - loss: 0.6556 - accuracy: 0.6319
Epoch 2/75
201/201 [==============================] - 0s 921us/step - loss: 0.5964 - accuracy: 0.6946
Epoch 3/75
201/201 [==============================] - 0s 898us/step - loss: 0.5824 - accuracy: 0.7074
Epoch 4/75
201/201 [==============================] - 0s 906us/step - loss: 0.5728 - accuracy: 0.7203
Epoch 5/75
201/201 [==============================] - 0s 952us/step - loss: 0.5684 - accuracy: 0.7239
Epoch 6/75
201/201 [==============================] - 0s 932us/step - loss: 0.5658 - accuracy: 0.7236
Epoch 7/75
201/201 [==============================] - 0s 883us/step - loss: 0.5643 - accuracy: 0.7250
Epoch 8/75
201/201 [==============================] - 0s 883us/step - loss: 0.5629 - accuracy: 0.7245
Epoch 9/75
201/201 [==============================] - 0s 878us/step - loss: 0.5617 - accuracy: 0.7250
Epoch 10/75
201/201 [==============================] - 0s 894us/step - loss: 0.560

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5470 - accuracy: 0.7348 - 268ms/epoch - 1ms/step
Loss: 0.5469850897789001, Accuracy: 0.734810471534729


In [18]:
# Export our model to HDF5 file
nn.save("output_data/AlphabetSoupCharity_Attempt_1.h5")

### Attempt 2

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 150
hidden_nodes_layer2 = 150
hidden_nodes_layer3 = 150
hidden_nodes_layer4 = 150
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third Hidden Layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth Hidden Layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 150)               6750      
                                                                 
 dense_5 (Dense)             (None, 150)               22650     
                                                                 
 dense_6 (Dense)             (None, 150)               22650     
                                                                 
 dense_7 (Dense)             (None, 150)               22650     
                                                                 
 dense_8 (Dense)             (None, 1)                 151       
                                                                 
Total params: 74851 (292.39 KB)
Trainable params: 74851 (292.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train, batch_size=128, epochs=100)

Epoch 1/100
201/201 [==============================] - 1s 2ms/step - loss: 0.5730 - accuracy: 0.7160
Epoch 2/100
201/201 [==============================] - 0s 2ms/step - loss: 0.5572 - accuracy: 0.7259
Epoch 3/100
201/201 [==============================] - 0s 2ms/step - loss: 0.5533 - accuracy: 0.7282
Epoch 4/100
201/201 [==============================] - 0s 2ms/step - loss: 0.5524 - accuracy: 0.7290
Epoch 5/100
201/201 [==============================] - 0s 2ms/step - loss: 0.5502 - accuracy: 0.7300
Epoch 6/100
201/201 [==============================] - 0s 2ms/step - loss: 0.5496 - accuracy: 0.7307
Epoch 7/100
201/201 [==============================] - 0s 2ms/step - loss: 0.5486 - accuracy: 0.7317
Epoch 8/100
201/201 [==============================] - 0s 2ms/step - loss: 0.5474 - accuracy: 0.7316
Epoch 9/100
201/201 [==============================] - 0s 2ms/step - loss: 0.5472 - accuracy: 0.7315
Epoch 10/100
201/201 [==============================] - 0s 2ms/step - loss: 0.5467 - accura

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5732 - accuracy: 0.7373 - 269ms/epoch - 1ms/step
Loss: 0.5732078552246094, Accuracy: 0.7372594475746155


In [23]:
# Export our model to HDF5 file
nn.save("output_data/AlphabetSoupCharity_Attempt_2.h5")

### Attempt 3

In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 50
hidden_nodes_layer2 = 75
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 50)                2250      
                                                                 
 dense_10 (Dense)            (None, 75)                3825      
                                                                 
 dense_11 (Dense)            (None, 1)                 76        
                                                                 
Total params: 6151 (24.03 KB)
Trainable params: 6151 (24.03 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train, batch_size=128, epochs=125)

Epoch 1/125
201/201 [==============================] - 1s 913us/step - loss: 1.6931 - accuracy: 0.6532
Epoch 2/125
201/201 [==============================] - 0s 928us/step - loss: 0.6795 - accuracy: 0.6983
Epoch 3/125
201/201 [==============================] - 0s 935us/step - loss: 0.7685 - accuracy: 0.6942
Epoch 4/125
201/201 [==============================] - 0s 904us/step - loss: 0.8121 - accuracy: 0.6932
Epoch 5/125
201/201 [==============================] - 0s 938us/step - loss: 0.8696 - accuracy: 0.6988
Epoch 6/125
201/201 [==============================] - 0s 931us/step - loss: 0.9265 - accuracy: 0.7088
Epoch 7/125
201/201 [==============================] - 0s 928us/step - loss: 0.9103 - accuracy: 0.7114
Epoch 8/125
201/201 [==============================] - 0s 932us/step - loss: 0.9480 - accuracy: 0.7065
Epoch 9/125
201/201 [==============================] - 0s 917us/step - loss: 1.6734 - accuracy: 0.6525
Epoch 10/125
201/201 [==============================] - 0s 893us/step - l

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5747 - accuracy: 0.7315 - 238ms/epoch - 888us/step
Loss: 0.5747327208518982, Accuracy: 0.7315452098846436


In [28]:
# Export our model to HDF5 file
nn.save("output_data/AlphabetSoupCharity_Attempt_3.h5")

### Attempt 4

In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 125
hidden_nodes_layer3 = 150
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="softmax"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 100)               4500      
                                                                 
 dense_13 (Dense)            (None, 125)               12625     
                                                                 
 dense_14 (Dense)            (None, 150)               18900     
                                                                 
 dense_15 (Dense)            (None, 1)                 151       
                                                                 
Total params: 36176 (141.31 KB)
Trainable params: 36176 (141.31 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [31]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train, batch_size=128, epochs=150)

Epoch 1/150
201/201 [==============================] - 1s 1ms/step - loss: 0.5812 - accuracy: 0.5316
Epoch 2/150
201/201 [==============================] - 0s 1ms/step - loss: 0.5617 - accuracy: 0.5316
Epoch 3/150
201/201 [==============================] - 0s 1ms/step - loss: 0.5579 - accuracy: 0.5316
Epoch 4/150
201/201 [==============================] - 0s 1ms/step - loss: 0.5573 - accuracy: 0.5316
Epoch 5/150
201/201 [==============================] - 0s 1ms/step - loss: 0.5548 - accuracy: 0.5316
Epoch 6/150
201/201 [==============================] - 0s 1ms/step - loss: 0.5531 - accuracy: 0.5316
Epoch 7/150
201/201 [==============================] - 0s 1ms/step - loss: 0.5524 - accuracy: 0.5316
Epoch 8/150
201/201 [==============================] - 0s 1ms/step - loss: 0.5533 - accuracy: 0.5316
Epoch 9/150
201/201 [==============================] - 0s 1ms/step - loss: 0.5516 - accuracy: 0.5316
Epoch 10/150
201/201 [==============================] - 0s 1ms/step - loss: 0.5511 - accura

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5654 - accuracy: 0.5349 - 259ms/epoch - 967us/step
Loss: 0.5654269456863403, Accuracy: 0.5349271297454834


In [33]:
# Export our model to HDF5 file
nn.save("output_data/AlphabetSoupCharity_Attempt_4.h5")

### Attempt 5

In [34]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 125
hidden_nodes_layer3 = 150
hidden_nodes_layer4 = 175
hidden_nodes_layer5 = 200
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 100)               4500      
                                                                 
 dense_17 (Dense)            (None, 125)               12625     
                                                                 
 dense_18 (Dense)            (None, 150)               18900     
                                                                 
 dense_19 (Dense)            (None, 175)               26425     
                                                                 
 dense_20 (Dense)            (None, 200)               35200     
                                                                 
 dense_21 (Dense)            (None, 1)                 201       
                                                                 
Total params: 97851 (382.23 KB)
Trainable params: 9785

In [35]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [36]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train, batch_size=128, epochs=175)

Epoch 1/175
201/201 [==============================] - 1s 2ms/step - loss: 0.5839 - accuracy: 0.7041
Epoch 2/175
201/201 [==============================] - 0s 2ms/step - loss: 0.5626 - accuracy: 0.7249
Epoch 3/175
201/201 [==============================] - 0s 2ms/step - loss: 0.5571 - accuracy: 0.7273
Epoch 4/175
201/201 [==============================] - 0s 2ms/step - loss: 0.5556 - accuracy: 0.7270
Epoch 5/175
201/201 [==============================] - 0s 2ms/step - loss: 0.5537 - accuracy: 0.7289
Epoch 6/175
201/201 [==============================] - 0s 2ms/step - loss: 0.5519 - accuracy: 0.7297
Epoch 7/175
201/201 [==============================] - 0s 2ms/step - loss: 0.5521 - accuracy: 0.7303
Epoch 8/175
201/201 [==============================] - 0s 2ms/step - loss: 0.5502 - accuracy: 0.7294
Epoch 9/175
201/201 [==============================] - 0s 2ms/step - loss: 0.5506 - accuracy: 0.7296
Epoch 10/175
201/201 [==============================] - 0s 2ms/step - loss: 0.5504 - accura

201/201 [==============================] - 0s 2ms/step - loss: 0.5326 - accuracy: 0.7383
Epoch 161/175
201/201 [==============================] - 0s 2ms/step - loss: 0.5324 - accuracy: 0.7386
Epoch 162/175
201/201 [==============================] - 0s 2ms/step - loss: 0.5321 - accuracy: 0.7386
Epoch 163/175
201/201 [==============================] - 0s 2ms/step - loss: 0.5322 - accuracy: 0.7389
Epoch 164/175
201/201 [==============================] - 0s 2ms/step - loss: 0.5315 - accuracy: 0.7390
Epoch 165/175
201/201 [==============================] - 0s 2ms/step - loss: 0.5314 - accuracy: 0.7389
Epoch 166/175
201/201 [==============================] - 0s 2ms/step - loss: 0.5318 - accuracy: 0.7387
Epoch 167/175
201/201 [==============================] - 0s 2ms/step - loss: 0.5326 - accuracy: 0.7394
Epoch 168/175
201/201 [==============================] - 0s 2ms/step - loss: 0.5321 - accuracy: 0.7390
Epoch 169/175
201/201 [==============================] - 0s 2ms/step - loss: 0.5319 - a

In [37]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5662 - accuracy: 0.7401 - 320ms/epoch - 1ms/step
Loss: 0.5662260055541992, Accuracy: 0.7400583028793335


In [38]:
# Export our model to HDF5 file
nn.save("output_data/AlphabetSoupCharity_Attempt_5.h5")

### Attempt 6

In [39]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 125
hidden_nodes_layer3 = 150
hidden_nodes_layer4 = 175
hidden_nodes_layer5 = 200
hidden_nodes_layer6 = 225
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="tanh"))

# Sixth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer6, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 100)               4500      
                                                                 
 dense_23 (Dense)            (None, 125)               12625     
                                                                 
 dense_24 (Dense)            (None, 150)               18900     
                                                                 
 dense_25 (Dense)            (None, 175)               26425     
                                                                 
 dense_26 (Dense)            (None, 200)               35200     
                                                                 
 dense_27 (Dense)            (None, 225)               45225     
                                                                 
 dense_28 (Dense)            (None, 1)                

In [40]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [41]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train, batch_size=128, epochs=200)

Epoch 1/200
201/201 [==============================] - 1s 2ms/step - loss: 0.5844 - accuracy: 0.7048
Epoch 2/200
201/201 [==============================] - 0s 2ms/step - loss: 0.5609 - accuracy: 0.7251
Epoch 3/200
201/201 [==============================] - 0s 2ms/step - loss: 0.5576 - accuracy: 0.7283
Epoch 4/200
201/201 [==============================] - 0s 2ms/step - loss: 0.5562 - accuracy: 0.7281
Epoch 5/200
201/201 [==============================] - 0s 2ms/step - loss: 0.5559 - accuracy: 0.7287
Epoch 6/200
201/201 [==============================] - 0s 2ms/step - loss: 0.5535 - accuracy: 0.7287
Epoch 7/200
201/201 [==============================] - 0s 2ms/step - loss: 0.5520 - accuracy: 0.7280
Epoch 8/200
201/201 [==============================] - 0s 2ms/step - loss: 0.5530 - accuracy: 0.7290
Epoch 9/200
201/201 [==============================] - 0s 2ms/step - loss: 0.5517 - accuracy: 0.7288
Epoch 10/200
201/201 [==============================] - 0s 2ms/step - loss: 0.5516 - accura

201/201 [==============================] - 0s 2ms/step - loss: 0.5318 - accuracy: 0.7393
Epoch 161/200
201/201 [==============================] - 0s 2ms/step - loss: 0.5321 - accuracy: 0.7394
Epoch 162/200
201/201 [==============================] - 0s 2ms/step - loss: 0.5320 - accuracy: 0.7388
Epoch 163/200
201/201 [==============================] - 0s 2ms/step - loss: 0.5320 - accuracy: 0.7388
Epoch 164/200
201/201 [==============================] - 0s 2ms/step - loss: 0.5315 - accuracy: 0.7392
Epoch 165/200
201/201 [==============================] - 0s 2ms/step - loss: 0.5325 - accuracy: 0.7385
Epoch 166/200
201/201 [==============================] - 0s 2ms/step - loss: 0.5320 - accuracy: 0.7389
Epoch 167/200
201/201 [==============================] - 0s 2ms/step - loss: 0.5319 - accuracy: 0.7387
Epoch 168/200
201/201 [==============================] - 0s 2ms/step - loss: 0.5331 - accuracy: 0.7386
Epoch 169/200
201/201 [==============================] - 0s 2ms/step - loss: 0.5331 - a

In [42]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5672 - accuracy: 0.7388 - 349ms/epoch - 1ms/step
Loss: 0.5672421455383301, Accuracy: 0.7387754917144775


In [43]:
# Export our model to HDF5 file
nn.save("output_data/AlphabetSoupCharity_Attempt_6.h5")